# 02 - Black-Scholes and Implied Volatility

Validate parity, compute Greeks, and recover implied volatility from synthetic prices.

In [6]:
%cd /content
!git clone https://github.com/basarr/interactive_portfolio_optimization.git


/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 59 (delta 14), reused 43 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 38.97 KiB | 1.11 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [7]:
from pathlib import Path
import sys

cands = [p.parent for p in Path("/content").rglob("pyproject.toml")]
print("pyproject parents:", cands)

ROOT = next((p for p in cands if (p / "src").exists()), None)
print("ROOT found:", ROOT)

if ROOT is None:
    raise FileNotFoundError("No project root with /src under /content. Clone/upload repo first.")

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("src exists:", (ROOT / "src").exists())


pyproject parents: [PosixPath('/content/interactive_portfolio_optimization')]
ROOT found: /content/interactive_portfolio_optimization
src exists: True


In [9]:
import numpy as np
import pandas as pd

from src.config import config_dict
from src.black_scholes import bs_call_price, bs_put_price, bs_delta, bs_gamma, bs_vega
from src.implied_vol import implied_vol
from src.plotting import plot_implied_vol_recovery

cfg = config_dict(fast_mode=True)
print("imports + cfg ready")

imports + cfg ready


In [10]:
strikes = np.linspace(80, 120, 9)
parity_rows = []
for K in strikes:
    C = bs_call_price(cfg['S0'], float(K), cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
    P = bs_put_price(cfg['S0'], float(K), cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
    rhs = cfg['S0'] * np.exp(-cfg['Q'] * cfg['T']) - float(K) * np.exp(-cfg['R'] * cfg['T'])
    parity_rows.append({'strike': K, 'parity_residual': C - P - rhs})
parity_df = pd.DataFrame(parity_rows)
parity_df.head()


,strike,parity_residual
0,80.0,3.552714e-15
1,85.0,7.105427e-15
2,90.0,0.000000e+00
3,95.0,-1.421085e-14
4,100.0,-7.105427e-15


In [11]:
sigma_true = cfg['SIGMA']
rows = []
for K in strikes:
    price = bs_call_price(cfg['S0'], float(K), cfg['R'], cfg['Q'], sigma_true, cfg['T'])
    iv = implied_vol(
        price=price,
        S=cfg['S0'],
        K=float(K),
        r=cfg['R'],
        q=cfg['Q'],
        T=cfg['T'],
        option_type='call',
    )
    rows.append({'strike': float(K), 'price': price, 'implied_vol': iv, 'sigma_true': sigma_true})

iv_df = pd.DataFrame(rows)
iv_df.to_csv(ROOT / 'results' / 'tables' / 'implied_vol_recovery.csv', index=False)
plot_implied_vol_recovery(iv_df, ROOT / 'results' / 'figures' / 'implied_vol_recovery.png')

greeks = {
    'delta_call': bs_delta(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'], 'call'),
    'gamma': bs_gamma(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T']),
    'vega': bs_vega(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T']),
}
print(greeks)
iv_df


{'delta_call': 0.579259709439103, 'gamma': 0.019552134698772795, 'vega': 39.104269397545586}


,strike,price,implied_vol,sigma_true
0,80.0,22.542853,0.2,0.2
1,85.0,18.469102,0.2,0.2
2,90.0,14.806507,0.2,0.2
3,95.0,11.613770,0.2,0.2
4,100.0,8.916037,0.2,0.2
5,105.0,6.704775,0.2,0.2
6,110.0,4.943867,0.2,0.2
7,115.0,3.578927,0.2,0.2
8,120.0,2.546926,0.2,0.2
